In [1]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
import os
from PIL import Image
import random

In [9]:
dataset_path = r'C:\Users\Siddhartha Devan V\Downloads\datasets\mnist_images\mnist_img_with_bb'
bb_path = r'C:\Users\Siddhartha Devan V\Downloads\datasets\mnist_images\mnist_img_enlarged_bb.csv'
labels = os.listdir(dataset_path)

In [21]:
annotations = pd.read_csv(bb_path)
annotations.head(10)

,Unnamed: 0,name,height,width,x_min,x_max,y_min,y_max
0,0,5_0.png,28,28,18,46,18,46
1,1,0_1.png,28,28,16,44,16,44
2,2,4_2.png,28,28,15,43,15,43
3,3,1_3.png,28,28,41,69,41,69
4,4,9_4.png,28,28,18,46,18,46
5,5,2_5.png,28,28,31,59,31,59
6,6,1_6.png,28,28,42,70,42,70
7,7,3_7.png,28,28,27,55,27,55
8,8,1_8.png,28,28,35,63,35,63
9,9,4_9.png,28,28,1,29,1,29


In [49]:
image_paths = []
bb = []
classes = []
s = 0
for i in labels:
    for j in annotations['name']:
        if int(j[0]) == int(i):
            label = j[0]
            bb_data = annotations[annotations['name']==j][['x_min', 'x_max', 'y_min', 'y_max']].values
            image_paths.append(os.path.join(dataset_path, label, j))
            bb.append(bb_data[0])
            classes.append(int(label))
            s+=1
        
        if s==10:
            break

KeyboardInterrupt: 